## Importing required modules

In [1]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import base64
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)
profile = service.users().getProfile(userId='me').execute()
profile

{'emailAddress': 'dongoodygoody@gmail.com',
 'messagesTotal': 3826,
 'threadsTotal': 3093,
 'historyId': '2864128'}

In [3]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)

In [4]:
filter = service.users().messages().list(userId = 'me', maxResults=10,
                                         q = 'from:no_reply@accessbankplc.com \
                                             subject:AccessAlert Transaction Alert').execute()

filter_id = filter['messages']
id_lst = []
for ids in filter_id:
    id_lst.append(ids['id'])
    

In [25]:
trans_lst = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    main_body = msgs['payload']['body']['data']
    main_body = main_body.replace('-', '+').replace('_', '/')
    decode_msg = base64.b64decode(main_body)
    soup = bs(decode_msg, 'lxml')
    details = soup.find_all('tr')[7]
    
    description = details.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
    reference_number = details.find_all('td')[8].text.replace('\r',' ').replace("\n"," ").strip()
    trans_branch = details.find_all('td')[10].text.replace('\r',' ').replace("\n"," ").strip()
    date = msgs['payload']['headers'][-1]['value']
    amount = re.search('\d*\.+\d+', snippet).group()
    acct_no = re.search('\d*\*+\d+', snippet).group()
    
    if 'Credited' in snippet:
        trans_type = 'Credited'
    else:
        trans_type = 'Debited'
    
    
    trans_lst.append({
        'amount': amount,
        'a/c_number': acct_no,
        'trans_type': trans_type,
        'description': description,
        'reference_number': reference_number,
        'trans_branch': trans_branch,
        'datetime': date
    })
cols_name = ['amount', 'a/c_number', 'trans_type', 'description', 'reference_number', 'trans_branch', 'datetime']
data = pd.DataFrame(trans_lst, columns=cols_name)

In [26]:
data

,amount,a/c_number,trans_type,description,reference_number,trans_branch,datetime
0,300.00,162******608,Debited,AIRTIME/ MTN/08168550974,099MJKL22253cxrA,HEAD OFFICE,"Sat, 10 Sep 2022 15:52:45 +0530"
1,200.00,162******608,Debited,AIRTIME/ MTN/08169327250,099MJKL22252PRIV,HEAD OFFICE,"Sat, 10 Sep 2022 00:36:44 +0530"
2,3000.00,070******357,Debited,POS/WEB PMT T_KCEE MAGNET GLOBA 006018 203,098WNVI222524j42,CENTRAL PROCESSING BRANCH,"Fri, 9 Sep 2022 12:41:36 +0530"
3,5000.00,070******357,Credited,TRF//FRM GOODRICH IFEANYI OKORO TO GOODRIC,099MJKL222522wGH,HEAD OFFICE,"Fri, 9 Sep 2022 12:36:09 +0530"
4,5000.00,162******608,Debited,TRF//FRM GOODRICH IFEANYI OKORO TO GOODRIC,099MJKL222522wGH,HEAD OFFICE,"Fri, 9 Sep 2022 12:36:08 +0530"
5,300.00,162******608,Debited,AIRTIME/ MTN/08168550974,099MJKL22251QAdF,HEAD OFFICE,"Fri, 9 Sep 2022 01:08:53 +0530"
6,100.00,162******608,Debited,AIRTIME/ GLO/08058825038,099MJKL22251KaT7,HEAD OFFICE,"Thu, 8 Sep 2022 21:54:20 +0530"
7,200.00,162******608,Debited,AIRTIME/ MTN/08169327250,099MJKL2224931pJ,HEAD OFFICE,"Tue, 6 Sep 2022 17:25:10 +0530"
8,200.00,162******608,Debited,AIRTIME/ MTN/08169327250,099MJKL22248NwKP,HEAD OFFICE,"Mon, 5 Sep 2022 03:34:59 +0530"
9,5100.00,070******357,Debited,POS/WEB PMT LA 00NG,098WNVI222472lM9,CENTRAL PROCESSING BRANCH,"Sun, 4 Sep 2022 22:35:43 +0530"


In [28]:
def extract_transaction():    
    trans_lst = []
    for each_id in id_lst:
        msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
        snippet = msgs['snippet']
        main_body = msgs['payload']['body']['data']
        main_body = main_body.replace('-', '+').replace('_', '/')
        decode_msg = base64.b64decode(main_body)
        soup = bs(decode_msg, 'lxml')
        details = soup.find_all('tr')[7]
        
        description = details.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
        reference_number = details.find_all('td')[8].text.replace('\r',' ').replace("\n"," ").strip()
        trans_branch = details.find_all('td')[10].text.replace('\r',' ').replace("\n"," ").strip()
        date = msgs['payload']['headers'][-1]['value']
        amount = re.search('\d*\.+\d+', snippet).group()
        acct_no = re.search('\d*\*+\d+', snippet).group()
        
        if 'Credited' in snippet:
            trans_type = 'Credited'
        else:
            trans_type = 'Debited'
        
        
        trans_lst.append({
            'amount': amount,
            'a/c_number': acct_no,
            'trans_type': trans_type,
            'description': description,
            'reference_number': reference_number,
            'trans_branch': trans_branch,
            'datetime': date
        })
    cols_name = ['amount', 'a/c_number', 'trans_type', 'description', 'reference_number', 'trans_branch', 'datetime']
    data = pd.DataFrame(trans_lst, columns=cols_name)
    return data

In [29]:
extract_transaction()

,amount,a/c_number,trans_type,description,reference_number,trans_branch,datetime
0,300.00,162******608,Debited,AIRTIME/ MTN/08168550974,099MJKL22253cxrA,HEAD OFFICE,"Sat, 10 Sep 2022 15:52:45 +0530"
1,200.00,162******608,Debited,AIRTIME/ MTN/08169327250,099MJKL22252PRIV,HEAD OFFICE,"Sat, 10 Sep 2022 00:36:44 +0530"
2,3000.00,070******357,Debited,POS/WEB PMT T_KCEE MAGNET GLOBA 006018 203,098WNVI222524j42,CENTRAL PROCESSING BRANCH,"Fri, 9 Sep 2022 12:41:36 +0530"
3,5000.00,070******357,Credited,TRF//FRM GOODRICH IFEANYI OKORO TO GOODRIC,099MJKL222522wGH,HEAD OFFICE,"Fri, 9 Sep 2022 12:36:09 +0530"
4,5000.00,162******608,Debited,TRF//FRM GOODRICH IFEANYI OKORO TO GOODRIC,099MJKL222522wGH,HEAD OFFICE,"Fri, 9 Sep 2022 12:36:08 +0530"
5,300.00,162******608,Debited,AIRTIME/ MTN/08168550974,099MJKL22251QAdF,HEAD OFFICE,"Fri, 9 Sep 2022 01:08:53 +0530"
6,100.00,162******608,Debited,AIRTIME/ GLO/08058825038,099MJKL22251KaT7,HEAD OFFICE,"Thu, 8 Sep 2022 21:54:20 +0530"
7,200.00,162******608,Debited,AIRTIME/ MTN/08169327250,099MJKL2224931pJ,HEAD OFFICE,"Tue, 6 Sep 2022 17:25:10 +0530"
8,200.00,162******608,Debited,AIRTIME/ MTN/08169327250,099MJKL22248NwKP,HEAD OFFICE,"Mon, 5 Sep 2022 03:34:59 +0530"
9,5100.00,070******357,Debited,POS/WEB PMT LA 00NG,098WNVI222472lM9,CENTRAL PROCESSING BRANCH,"Sun, 4 Sep 2022 22:35:43 +0530"


In [9]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
e

'Amt includes COMM & VAT/USSD/GOODRICH IFEA'

In [ ]:
payload = msgs['payload']
headers = payload['headers']

parts = payload.get('body')['data']
parts = parts.replace('-','+').replace('_','/')
decode = base64.b64decode(parts)
soup = bs(decode,"lxml" )
body = soup.body()

In [150]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
e

'Amt includes COMM & VAT/USSD/GOODRICH IFEA'

In [44]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[8].text
e

'\r\n                                            099MJNL2224484GU\r\n                                        '

In [46]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[10].text
e

'\r\n                                            HEAD OFFICE\r\n                                        '

In [ ]:
pat = re.compile(r'\d*\.?\d+')
for x in msg_lst:
   pad = re.findall('\d*\.?\d+', x)

print(pad[0])

11200.00


In [ ]:
import re
bank = []
pattern = re.compile(r'Debited')
b = you if re.search('Debited', you) else 3
print(b)

# import re
# pattern=re.compile(r'ack') 
# str_match = [x for x in my_list if re.search('ack', x)]
# print(str_match)

Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
